## Environment Setup

In [37]:
# If packages are not installed
!pip install -r requirements.txt

In [39]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
!pip install autoviz

  Using cached https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached visions-0.8.1-py3-none-any.whl.metadata (11 kB)
  Using cached htmlmin-0.1.12.tar.gz (19 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [1 lines of output]
  ERROR: Can not execute `setup.py` since setuptools is not available in the build environment.
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [40]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from datetime import datetime
from pathlib import Path
from pandas_profiling import ProfileReport
from IPython.display import display

from autoviz.AutoViz_Class import AutoViz_Class

# Suppress the warning
warnings.filterwarnings('ignore')

# Set display config
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

ModuleNotFoundError: No module named 'pandas_profiling'

In [12]:
from config.paths import RAW_DATA_DIR

MARKET_DATA_PATH = RAW_DATA_DIR / "long-btc-market-data.csv"
NETWORK_DATA_PATH = RAW_DATA_DIR / "long-btc-network-data.csv"
MARKET_INDICATOR_PATH = RAW_DATA_DIR / "long-btc-market-indicator.csv"
NETWORK_INDICATOR_PATH = RAW_DATA_DIR / "long-btc-network-indicator.csv"
FLOW_INDICATOR_PATH = RAW_DATA_DIR / "long-btc-flow-indicator.csv"
EXCHANGE_FLOWS_PATH = RAW_DATA_DIR / "long-btc-exchange-flows.csv"
MINER_FLOWS_PATH = RAW_DATA_DIR / "long-btc-miner-flows.csv"

# Exchange Flows

## Definition 

1. **reserve**: The total amount of coins held in the exchange addresses.

2. **reserve_usd**: The total amount of coins (in USD) held in exchange addresses.

3. **netflow_total**: The total diffference between BTC flowing into and out of the exchange (Inflow - Outflow).

4. **inflow_mean**: The mean amount of BTC per transaction sent to the exchange.

5. **inflow_mean_ma7**: The 7 days moving average of mean BTC inflow to the exchange.

6. **inflow_top10**: The total BTC amount of the top 10 transactions inflow to the exchange.

7. **inflow_total**: The total amount of BTC transferred to the exchange.

8. **outflow_mean**: The mean amount of BTC per transaction sent from the exchange.

9. **outflow_mean_ma7**: The 7 days moving average of mean BTC outflow from the exchange.

10. **outflow_top10**: 
The total BTC amount of the top 10 transactions outflow from the exchange

11. **outflow_total**: The total amount of BTC transferred from the exchange.

12. **transactions_count_inflow**: The total number of deposits transactions to the exchange.

13. **transactions_count_outflow**: 
The total number of withdrawals from the exchange

14. **addresses_count_inflow**: 
The number of the token addresses making inflow transactions to the exchange

15. **addresses_count_outflow**: The number of the token addresses making outflow transactions from the exchange.


## Interpretation

### 1. Exchange Reserve

**Exchange Reserve** is a key metric in determining the availability of coins for potential sale within the market and provide insights into market dynamics. 

An **upward trend in exchange reserves typically suggests an increasing selling pressure**, indicating that more coins from the exchange's holdings are potentially entering the market for sale (market participants are opting to keep a large portion of their holdings within the exchange, suggesting a readiness to sell).

Conversely, a **downward trend in exchange reserves points towards decreasing selling pressure**, implying that fewer coins are available for sale (market participants are more inclined to withdraw their coins from the exchange for purposes other than immediate selling).

#### Examining Trends

**Increasing Trend** --> Decrease in currency scarcity --> **Bearish**

**Decreasing Trend** --> Increasing scarcity of coins --> **Bullish**

### 2. Exchange In/Outflow and Netflow

In [21]:
# Read the CSV file
try:
    exchange_flows_data = pd.read_csv(MARKET_DATA_PATH)
    if exchange_flows_data is not None:
        print("File loaded successfully")
except IOError as e:
    print("File Not Found")

File loaded successfully


In [22]:
exchange_flows_data.head()

,date,close,high,low,open,volume,open_interest,taker_buy_ratio,taker_buy_sell_ratio,taker_buy_volume,taker_sell_ratio,taker_sell_volume,long_liquidations,long_liquidations_usd,short_liquidations,short_liquidations_usd,coinbase_premium_gap,coinbase_premium_gap_usdt_adjusted,coinbase_premium_index,coinbase_premium_index_usdt_adjusted
0,2009-01-03,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-01-04,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-01-05,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-01-06,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-01-07,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
exchange_flows_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5920 entries, 0 to 5919
Data columns (total 20 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   date                                  5920 non-null   object 
 1   close                                 5920 non-null   float64
 2   high                                  5920 non-null   float64
 3   low                                   5920 non-null   float64
 4   open                                  5920 non-null   float64
 5   volume                                5920 non-null   float64
 6   open_interest                         2182 non-null   float64
 7   taker_buy_ratio                       3463 non-null   float64
 8   taker_buy_sell_ratio                  3463 non-null   float64
 9   taker_buy_volume                      3463 non-null   float64
 10  taker_sell_ratio                      3463 non-null   float64
 11  taker_sell_volume